In [1]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class CustomDataset(Dataset):
    def __init__(self, data_dir, image_files, annotations, transform=None):
        self.data_dir = data_dir
        self.image_files = image_files
        self.annotations = annotations
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.data_dir, 'images', self.image_files[idx])
        image = Image.open(image_path).convert('RGB')

        filename = self.image_files[idx]
        annotation = self.annotations.get(filename, {})

        if self.transform is not None:
            image = self.transform(image)

        return image, annotation

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

images_dir = 'dataset/images/'
masks_dir = 'dataset/masks/'

In [4]:
# List all files in the directory and filter for image file extensions (e.g., '.jpg', '.png')
image_files = [file for file in os.listdir(images_dir) if file.lower().endswith(('.jpg', '.png'))]

# Now image_files contains a list of image filenames from the "dataset/images" directory
print(image_files)

['Screenshot_1.png', 'Screenshot_10.png', 'Screenshot_100.png', 'Screenshot_101.png', 'Screenshot_102.png', 'Screenshot_103.png', 'Screenshot_104.png', 'Screenshot_105.png', 'Screenshot_106.png', 'Screenshot_108.png', 'Screenshot_109.png', 'Screenshot_11.png', 'Screenshot_110.png', 'Screenshot_111.png', 'Screenshot_112.png', 'Screenshot_12.png', 'Screenshot_13.png', 'Screenshot_14.png', 'Screenshot_15.png', 'Screenshot_16.png', 'Screenshot_17.png', 'Screenshot_18.png', 'Screenshot_19.png', 'Screenshot_2.png', 'Screenshot_20.png', 'Screenshot_21.png', 'Screenshot_22.png', 'Screenshot_23.png', 'Screenshot_24.png', 'Screenshot_25.png', 'Screenshot_26.png', 'Screenshot_27.png', 'Screenshot_28.png', 'Screenshot_29.png', 'Screenshot_3.png', 'Screenshot_30.png', 'Screenshot_31.png', 'Screenshot_32.png', 'Screenshot_33.png', 'Screenshot_34.png', 'Screenshot_35.png', 'Screenshot_36.png', 'Screenshot_37.png', 'Screenshot_38.png', 'Screenshot_39.png', 'Screenshot_4.png', 'Screenshot_40.png', 'Scr

In [5]:
annotations_dict = {
    "Screenshot_1.png": [912, 411, 1022, 543],
    "Screenshot_10.png": [180, 142, 408, 383],
    "Screenshot_100.png": [202, 226, 300, 348],
    "Screenshot_101.png": [266, 159, 369, 285],
    "Screenshot_102.png": [334, 275, 431, 411],
    "Screenshot_103.png": [386, 314, 501, 474],
    "Screenshot_104.png": [707, 373, 796, 522],
    "Screenshot_105.png": [852, 201, 950, 315],
    "Screenshot_106.png": [669, 258, 718, 330],
    "Screenshot_108.png": [534, 193, 626, 298],
    "Screenshot_109.png": [476, 205, 569, 321],
    "Screenshot_11.png": [185, 140, 1051, 756],
    "Screenshot_110.png": [255, 361, 360, 475],
    "Screenshot_111.png": [320, 373, 412, 479],
    "Screenshot_112.png": [406, 415, 503, 532],
    "Screenshot_12.png": [671, 284, 1036, 530],
    "Screenshot_13.png": [514, 113, 614, 523],
    "Screenshot_14.png": [421, 628, 584, 781],
    "Screenshot_15.png": [862, 270, 932, 354],
    "Screenshot_16.png": [112, 334, 211, 462],
    "Screenshot_17.png": [777, 95, 834, 174],
    "Screenshot_18.png": [581, 133, 632, 213],
    "Screenshot_19.png": [667, 77, 742, 143],
    "Screenshot_2.png": [798, 475, 871, 579],
    "Screenshot_20.png": [688, 424, 769, 511],
    "Screenshot_21.png": [755, 187, 810, 241],
    "Screenshot_22.png": [524, 171, 570, 221],
    "Screenshot_23.png": [405, 416, 472, 486],
    "Screenshot_24.png": [459, 211, 515, 284],
    "Screenshot_25.png": [1415, 397, 1475, 475],
    "Screenshot_26.png": [521, 290, 647, 468],
    "Screenshot_27.png": [523, 396, 625, 496],
    "Screenshot_28.png": [185, 198, 1268, 443],
    "Screenshot_29.png": [345, 104, 593, 433],
    "Screenshot_3.png": [288, 336, 406, 475],
    "Screenshot_30.png": [140, 145, 1797, 337],
    "Screenshot_31.png": [339, 252, 428, 352],
    "Screenshot_32.png": [922, 218, 999, 316],
    "Screenshot_33.png": [1138, 374, 1233, 490],
    "Screenshot_34.png": [1176, 263, 1235, 328],
    "Screenshot_35.png": [161, 198, 229, 260],
    "Screenshot_36.png": [197, 128, 258, 182],
    "Screenshot_37.png": [671, 172, 729, 231],
    "Screenshot_38.png": [314, 183, 1460, 382],
    "Screenshot_39.png": [226, 180, 1029, 705],
    "Screenshot_4.png": [383, 326, 447, 410],
    "Screenshot_40.png": [285, 238, 359, 307],
    "Screenshot_41.png": [328, 293, 412, 378],
    "Screenshot_42.png": [991, 375, 1092, 481],
    "Screenshot_43.png": [935, 217, 1003, 281],
    "Screenshot_44.png": [266, 143, 343, 198],
    "Screenshot_45.png": [483, 264, 550, 335],
    "Screenshot_46.png": [885, 255, 942, 314],
    "Screenshot_47.png": [1226, 125, 1282, 188],
    "Screenshot_48.png": [643, 417, 707, 499],
    "Screenshot_49.png": [421, 269, 479, 332],
    "Screenshot_5.png": [72, 301, 155, 405],
    "Screenshot_50.png": [604, 233, 666, 291],
    "Screenshot_51.png": [1305, 244, 1372, 317],
    "Screenshot_52.png": [543, 97, 588, 161],
    "Screenshot_53.png": [787, 324, 845, 414],
    "Screenshot_54.png": [613, 288, 737, 376],
    "Screenshot_55.png": [585, 359, 684, 481],
    "Screenshot_56.png": [977, 213, 1015, 271],
    "Screenshot_57.png": [215, 402, 254, 461],
    "Screenshot_58.png": [942, 113, 1001, 183],
    "Screenshot_59.png": [396, 287, 465, 354],
    "Screenshot_6.png": [122, 106, 844, 673],
    "Screenshot_60.png": [540, 236, 653, 337],
    "Screenshot_61.png": [896, 284, 1020, 437],
    "Screenshot_62.png": [761, 318, 995, 591],
    "Screenshot_63.png": [40, 203, 1397, 404],
    "Screenshot_64.png": [72, 150, 1464, 348],
    "Screenshot_65.png": [253, 221, 1333, 509],
    "Screenshot_66.png": [633, 114, 699, 171],
    "Screenshot_68.png": [659, 213, 715, 288],
    "Screenshot_69.png": [13, 443, 77, 515],
    "Screenshot_7.png": [246, 163, 1120, 555],
    "Screenshot_70.png": [1053, 216, 1102, 273],
    "Screenshot_71.png": [15, 154, 114, 231],
    "Screenshot_72.png": [584, 157, 695, 248],
    "Screenshot_73.png": [794, 489, 923, 691],
    "Screenshot_74.png": [966, 237, 1005, 288],
    "Screenshot_75.png": [1016, 278, 1074, 337],
    "Screenshot_76.png": [575, 267, 633, 331],
    "Screenshot_77.png": [1172, 329, 1255, 423],
    "Screenshot_78.png": [1114, 350, 1226, 482],
    "Screenshot_79.png": [127, 577, 306, 793],
    "Screenshot_8.png": [17, 324, 104, 420],
    "Screenshot_80.png": [968, 215, 1073, 305],
    "Screenshot_81.png": [1059, 213, 1163, 312],
    "Screenshot_82.png": [194, 198, 294, 553],
    "Screenshot_83.png": [300, 473, 422, 594],
    "Screenshot_84.png": [57, 398, 166, 508],
    "Screenshot_85.png": [254, 284, 339, 369],
    "Screenshot_86.png": [203, 505, 349, 637],
    "Screenshot_87.png": [263, 254, 401, 373],
    "Screenshot_88.png": [452, 309, 525, 383],
    "Screenshot_89.png": [207, 220, 308, 302],
    "Screenshot_9.png": [45, 149, 1140, 450],
    "Screenshot_90.png": [139, 381, 232, 468],
    "Screenshot_91.png": [630, 170, 760, 295],
    "Screenshot_92.png": [971, 269, 1122, 409],
    "Screenshot_93.png": [856, 427, 990, 594],
    "Screenshot_94.png": [275, 311, 379, 457],
    "Screenshot_95.png": [514, 169, 637, 291],
    "Screenshot_96.png": [208, 158, 323, 296],
    "Screenshot_97.png": [490, 173, 631, 290],
    "Screenshot_98.png": [644, 266, 764, 414],
    "Screenshot_99.png": [499, 109, 618, 208]
}

annotations = {}  # Reset the annotations dictionary

# Populate the annotations dictionary with "boxes" key
for filename, annotation in annotations_dict.items():
    x1, y1, x2, y2 = annotation
    width = x2 - x1
    height = y2 - y1
    image_index = image_files.index(filename)  # Get the index of the current image
    annotations[filename] = {
        'boxes': torch.tensor([[x1, y1, x2, y2]], dtype=torch.float32),  # Add the "boxes" key
        'labels': torch.tensor([1], dtype=torch.int64),  # Assuming there's a single class (e.g., '1') for all objects
        'image_id': torch.tensor([image_index]),  # Use the image index as the unique identifier
        'area': torch.tensor([width * height], dtype=torch.float32),
        'iscrowd': torch.tensor([0], dtype=torch.int64),  # Assuming all instances are not crowd
    }

# Now annotations is in the required format
print(annotations)

{'Screenshot_1.png': {'boxes': tensor([[ 912.,  411., 1022.,  543.]]), 'labels': tensor([1]), 'image_id': tensor([0]), 'area': tensor([14520.]), 'iscrowd': tensor([0])}, 'Screenshot_10.png': {'boxes': tensor([[180., 142., 408., 383.]]), 'labels': tensor([1]), 'image_id': tensor([1]), 'area': tensor([54948.]), 'iscrowd': tensor([0])}, 'Screenshot_100.png': {'boxes': tensor([[202., 226., 300., 348.]]), 'labels': tensor([1]), 'image_id': tensor([2]), 'area': tensor([11956.]), 'iscrowd': tensor([0])}, 'Screenshot_101.png': {'boxes': tensor([[266., 159., 369., 285.]]), 'labels': tensor([1]), 'image_id': tensor([3]), 'area': tensor([12978.]), 'iscrowd': tensor([0])}, 'Screenshot_102.png': {'boxes': tensor([[334., 275., 431., 411.]]), 'labels': tensor([1]), 'image_id': tensor([4]), 'area': tensor([13192.]), 'iscrowd': tensor([0])}, 'Screenshot_103.png': {'boxes': tensor([[386., 314., 501., 474.]]), 'labels': tensor([1]), 'image_id': tensor([5]), 'area': tensor([18400.]), 'iscrowd': tensor([0]

In [ ]:
#concatenate images with masks and tensor coordinates of bounding boxes
